<a href="https://colab.research.google.com/github/sanspareilsmyn/mldl_sandbox/blob/main/transfer_learning_cnn_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://github.com/WillKoehrsen/pytorch_challenge/blob/master/Transfer%20Learning%20in%20PyTorch.ipynb

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
import seaborn as nss
# PyTorch
from torchvision import transforms, datsets, models
from torch import optim, cuda
from torch.utils.data import DataLoader, sampler
import torch.nn as nn

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# Data science tools
import numpy as np
import pandas as pd
import os

# Image manipulations
from PIL import Image

# Useful for examining network
from torchsummary import summary

# Timing utility
from timeit import default_timer as timer

# Visualizations
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.size'] = 14

# Printing out all outputs
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
datadir = '/home/wjk68/'
traindir = datadir + 'train/'
validdir = datadir + 'valid/'
testdir = datadir + 'test/'

save_file_name = 'vgg16-transfer-4.pt'
checkpoint_path = 'vgg16-transfer-4.pth'

batch_size = 128

train_on_gpu = cuda.is_available()
print(f'Train on gpu: {train_on_gpu}')

# Number of gpus
if train_on_gpu:
  gpu_count = cuda.device_count()
  print(f'{gpu_count} gpus detected.')
  if gpu_count > 1:
    multi_gpu = True
  else:
    multi_gpu = False

# Data Exploration

In [ ]:
# Empty lists
categories = []
img_cateogires = []
n_train = []
n_valid = []
n_test = []
hs = []
ws = []

# Iterate through each category
for d in os.listdir(traindir):
  categories.append(d)

  # Number of each image
  train_imgs = os.listdir(traindir + d)
  valid_imgs = os.listdir(validdir + d)
  test_imgs = os.listdir(testdir + d)
  n_train.append(len(train_imgs))
  n_valid.append(len(valid_imgs))
  n_test.append(len(test_imgs))

  # Find stats for train images
  for i in train_imgs:
    img_categories.append(d)
    img = Image.open(traindir + d + '/' + i)
    img_array = np.array(img)
    # Shape
    hs.append(img_array.shape[0])
    ws.append(img_array.shape[1])

# Dataframe of categories
cat_df = pd.DataFrame({'category' : categories,
                       'n_train' : n_train,
                       'n_valid' : n_valid,
                       'n_test' : n_test}).sort_values('category')

image_df = pd.DataFrame({
    'category' : img_categories,
    'height' : hs,
    'width' : ws
})

cat_df.sort_values('n_train', ascending=False, inplace=True)
cat_df.head()
cat_df.tail()

In [ ]:
cat_df.set_index('category')['n_train'].plot.bar(
    color='r', figsize=(20, 6))
plt.xticks(rotation=80)
plt.ylabel('Count')
plt.title('Training Images by Category')

In [ ]:
cat_df.set_index('category').iloc[:50]['n_train'].plot.bar(
    color='r', figsize=(20, 6))
plt.xticks(rotation=80)
plt.ylabel('Count')
plt.title('Training Images by Category')

# Image Preprocessing

In [3]:
# Data Augmentation

In [ ]:
image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards     
    ]),
    # Validation does not use augmentation
    'val':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards     
    ]),
    # Test does not use augmentation
    'test':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [4]:
# Data Iterators

In [ ]:
# Datasets from each folder
data = {
    'train':
    datasets.ImageFolder(root=traindir, transform=image_transforms['train']),
    'val':
    datasets.ImageFolder(root=validdir, transform=image_transforms['val']),
    'test':
    datasets.ImageFolder(root=testdir, transform=image_transform['test'])
}

# Dataloader iterators
dataloaders = {
    'train':
    DataLoader(data['train'], batch_size=batch_size, shuffle=True),
    'val':
    DataLoader(data['val'], batch_size=batch_size, shuffle=True),
    'test':
    DataLoader(data['test'], batch_size=batch_size, shuffle=True)
}

In [ ]:
trainiter = iter(dataloaders['train'])
features, labels = next(trainiter)
features.shape, labels.shape

In [ ]:
n_classes = len(cat_df)
print(f'There are {n_classes} different classes.')

len(data['train'].classes)

# Pre-trained Models for Image Classification

In [ ]:
model = models.vgg16(pretrained=True)
model

In [ ]:
# Freeze Early layers
for param in model.parameters():
  parm.requires_grad = False

In [ ]:
n_inputs = model.classifier[6].in_features

# Add on classifier
model.classifier[6] = nn.Sequential(
    nn.Linear(n_inputs, 256), nn.ReLU(), nn.Dropout(0.4),
    nn.Linear(256, n_classes), nn.LogSoftmax(dim=1))

model.classifier

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

In [ ]:
if train_on_gpu:
    model = model.to('cuda')

if multi_gpu:
    model = nn.DataParallel(model)

# Function to Load in Pretrained Model

In [ ]:
def get_pretrained_model(model_name):
  '''
  Retrieve a pre-trained model from torchvision

  Params
  -------
  Model_name(str): name of the model(currently only accpets vgg16 and resnet50)

  Return
  ------
  model (PyTorch Model) : cnn
  '''

  if model_name == 'vgg16':
    model = models.vgg16(pretrained=True)

    # Freeze early layers
    for param in model.parameters():
      param.requires_grad = False
    n_inputs = model.classifier[6].in_features

    # Add on classifier
    model.classifier[6] = nn.Sequential(
        nn.Linear(n_inputs, 256), nn.ReLU(), nn.Dropout(0.2),
        nn.Linear(256, n_classes), nn.LogSoftMax(dim=1)
    )

  elif model_name == 'resnet50':
    model = models.resnet50(pretrained=True)

    # Freeze early layers
    for param in model.parameters():
      param.requires_grad = False

    # Add on classifier
    model.fc = nn.Sequential(
        nn.Linear(n_inputs, 256), nn.ReLU(), nn.Dropout(0.2),
        nn.Linear(256, n_classes), nn.LogSoftmax(dim=1)
    )

  if train_on_gpu:
    model = model.to('cuda')
  
  if multi_gpu:
    model = nn.DataParallel(model)

  return model

In [ ]:
model = get_pretrained_model('vgg16')
if multi_gpu:
  summary(
      model.module,
      input_size=(3, 224, 224),
      batch_size=batch_size,
      device='cuda'
  )
else:
  summary(
      model,
      input_size=(3, 224, 224),
      batch_size=batch_size,
      device='cuda'
  )

In [ ]:
# Mapping of Classes to Indexes
model.class_to_idx = data['train'].class_to_idx
model.idx_to_class = {
    idx: class_ for class_, idx in model.class_to_idx.items()
}
list(model.idx_to_class.items())[:10]

# Training Loss and Optimizer

In [ ]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
for p in optimizer.param_groups[0]['params']:
  if p.requires_grad:
    print(p.shape)

In [ ]:
def train(model, criterion, optimizer, train_loader, valid_loader, save_file_name, max_epochs_stop=3, n_epochs=20, print_every=2):
  '''
  Train a PyTorch Model

  Params
  -------
  model (PyTorch model) : cnn to train
  criterion (PyTorch loss) : objective to minimize
  optimizer (PyTorch optimizer) : optimizer to compute gradients of model parameters
  train_loader (PyTorch dataloader) : training dataloader to iterate through
  valid_loader (PyTorch dataloader) : validation dataloader used for early stopping
  save_file_name (str ending in '.pt') : file path to save the model state dict
  max_epochs_stop (int) : maximum number of epochs with no improvement in validation loss for early stopping
  print_every (int) : frequency of epochs to print training stats

  Returns
  -------
  model (PyTorch model) : trained cnn with best weights
  history (DataFrame) : history of train and validation loss and accuracy
  '''

  # Early stopping initialization
  epochs_no_improve = 0
  valid_loss_min = np.Inf

  valid_max_acc = 0
  history = []

  # Number of epocs already trained (if using loaded in model weights)
  try:
    print(f'Model has been trained for: {model.epochs} epochs.\n')
  except:
    model.epochs = 0
    print(f'Starting Training from Scratch.\n')
  overall_start = timer()

  # Main Loop
  for epoch in range(n_epochs):
    # Keep track of training and validation loss each epoch
    train_loss = 0.0
    valid_loss = 0.0
    
    train_acc = 0
    valid_acc = 0

    # Training Loop
    for ii, (data, target) in enumerate(train_loader):
      # Tensors to gpu
      if train_on_gpu:
        data, target = data.cuda(), target.cuda()
      
      # Clear gradients
      optimizer.zero_grad()
      output = model(data)

      # Loss and Backpropagation of gradients
      loss = criterion(output, target)
      loss.backward()

      # Update the parameters
      optimizer.step()

      # Track train loss by multipying average loss by number of examples in batch
      train_loss += loss.item() * data.size(0)

      # Calculate accracy by finding max log probability
      _, pred = torch.max(output, dim=1)
      correct_tensor = pred.eq(target.data.view_as(pred))

      # Need to convert correct tensor from int to float to average
      accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))

      # Multipy average accuracy times the number of examples in batch
      train_acc += accuracy.item() * data.size(0)

      # Track training progress
      print(f'Epoch: {epoch}\t{100 * (ii + 1) / len(train_loader):.2f}% complete. {timer() - start:.2f} seconds elapsed in epoch.', end='\r')
    
    # After training loops ends, start validation
    else:
      model.epochs += 1
        # Don't need to keep track of gradients
        with torch.no_grad():
          # Set to evaluation mode
          model.eval()

          # Validation loop
          for data, target in valid_loader:
            if train_on_gpu:
              data, target = data.cuda(), target.cuda()
            
            # Forward pass
            output = model(data)

            # Validation loss
            loss = criterion(output, target)
            
